In [18]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from pyod.models.iforest import IForest
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import OneSidedSelection
import os
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from pyod.models.pca import PCA
from sklearn.mixture import GaussianMixture
from sklearn.covariance import EllipticEnvelope
from sklearn.model_selection import GridSearchCV

import os, sys, pickle
import argparse, sys

import seaborn as sns
sns.set_style('white')

import warnings
warnings.filterwarnings('ignore')



# parser = argparse.ArgumentParser()
# parser.add_argument('-label', help=' : Please set the label') 
# args = parser.parse_args()


df = pd.read_csv('data/modified_dataset.csv')


# def remove_outliers(df, column_name, factor=1.5):
#     Q1 = df[column_name].quantile(0.25)
#     Q3 = df[column_name].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - factor * IQR
#     upper_bound = Q3 + factor * IQR
#     df_filtered = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
#     return df_filtered

# # 모든 칼럼에 대해 이상치 제거 실행
# for column_name in df.columns:
#     df = remove_outliers(df, column_name)







df_stroke_0 = df[df['Cancer'] == 0]
df_stroke_1 = df[df['Cancer'] == 1]



#train, test data processing 1,2,3,4중 선택

#1.정상으로만 학습, 테스트는 정상과 이상 반반
answer_label = 'Cancer'
# X = df[df.columns.difference([answer_label])]
# df_normal = df[df[answer_label] == 0]
# df_abnormal = df[df[answer_label] == 1]
# test_normal_df = df_normal.sample(n=len(df_stroke_1), random_state = 0)
# test_df = pd.concat([df_abnormal, test_normal_df])
# X_test = test_df[test_df.columns.difference([answer_label])]
# y_test = test_df[answer_label]
# train_df = df_normal.drop(test_normal_df.index)
# X_train = train_df[train_df.columns.difference([answer_label])]
# y_train = train_df[answer_label]

#그냥데이터
# X = df.drop(answer_label,axis=1)
# y = df[answer_label]
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)

#3.Oversampling by Smote
overs = SMOTE(random_state=1)
X = df.drop(answer_label,axis=1)
y = df[answer_label]
X_os, y_os = overs.fit_resample(X,y)
X_train,X_test,y_train,y_test = train_test_split(X_os,y_os,test_size=0.3,random_state=1)



In [19]:
def main_task(n_neighbors, weights, algorithm, metric):
    knn = KNeighborsClassifier(n_neighbors = n_neighbors, weights=weights,
                               algorithm='brute', metric=metric)
    knn.fit(X_train, y_train)
    knn_test_pred = knn.predict(X_test)
    return round(f1_score(knn_test_pred, y_test),3)

In [20]:
knn = KNeighborsClassifier(n_neighbors = 4, weights='distance',
                            algorithm='brute', metric='minkowski')
knn.fit(X_train, y_train)
knn_test_pred = knn.predict(X_test)
print(round(f1_score(knn_test_pred, y_test),3))

0.961


In [21]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

def optuna_main(trial: optuna.Trial):
    n_neighbors = trial.suggest_int('n_neighbors', 5, 20, step=1)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])
    metric = trial.suggest_categorical('metric', ['minkowski', 'euclidean', 'manhattan'])

    f1_score = main_task(n_neighbors=n_neighbors, weights=weights, algorithm=algorithm, metric=metric)

    print('<f1_score:', f1_score,'>')
    return f1_score

study = optuna.create_study(direction='maximize')
study.optimize(optuna_main, n_trials=100)
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_params : {study.best_trial.params}')

[I 2023-11-08 15:33:35,075] A new study created in memory with name: no-name-0545685e-578f-4e57-a580-0513f3108014
[I 2023-11-08 15:33:35,529] Trial 0 finished with value: 0.945 and parameters: {'n_neighbors': 15, 'weights': 'uniform', 'algorithm': 'brute', 'metric': 'euclidean'}. Best is trial 0 with value: 0.945.
[I 2023-11-08 15:33:35,715] Trial 1 finished with value: 0.955 and parameters: {'n_neighbors': 7, 'weights': 'distance', 'algorithm': 'brute', 'metric': 'euclidean'}. Best is trial 1 with value: 0.955.


<f1_score: 0.945 >
<f1_score: 0.955 >


[I 2023-11-08 15:33:36,309] Trial 2 finished with value: 0.96 and parameters: {'n_neighbors': 6, 'weights': 'distance', 'algorithm': 'brute', 'metric': 'manhattan'}. Best is trial 2 with value: 0.96.


<f1_score: 0.96 >


[I 2023-11-08 15:33:36,852] Trial 3 finished with value: 0.941 and parameters: {'n_neighbors': 20, 'weights': 'uniform', 'algorithm': 'ball_tree', 'metric': 'euclidean'}. Best is trial 2 with value: 0.96.


...

<f1_score: 0.949 >


[I 2023-11-08 15:34:25,646] Trial 98 finished with value: 0.959 and parameters: {'n_neighbors': 5, 'weights': 'distance', 'algorithm': 'brute', 'metric': 'manhattan'}. Best is trial 2 with value: 0.96.


<f1_score: 0.959 >


[I 2023-11-08 15:34:26,276] Trial 99 finished with value: 0.96 and parameters: {'n_neighbors': 6, 'weights': 'distance', 'algorithm': 'brute', 'metric': 'manhattan'}. Best is trial 2 with value: 0.96.


<f1_score: 0.96 >
study.best_trial.value : 0.96
study.best_params : {'n_neighbors': 6, 'weights': 'distance', 'algorithm': 'brute', 'metric': 'manhattan'}


In [22]:
#결과
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_paramse : {study.best_params}')

study.best_trial.value : 0.96
study.best_paramse : {'n_neighbors': 6, 'weights': 'distance', 'algorithm': 'brute', 'metric': 'manhattan'}
